In [300]:
import sys
!{sys.executable} -m pip install bs4
import requests
from pandas.io.json import json_normalize
import json
from datetime import datetime
from datetime import timedelta
import feedparser
import pandas as pd

In [332]:
def scrapping(soup):
    #Obtengo los principales índices para el cierre anterior (es una tabla)
    tabla_indices = soup.select("table table table table table th")
    # tabla_indices
    headers = [element.text for element in tabla_indices[0:10]]

    # Guardo los nombres de la columna para la tabla
    headers

    #Obtengo todos los datos que se encuentran en la tabla, en los tabledata
    datos  = soup.select("table table table table table td")

    #Tomo el texto de cada elemento y los guardo, sin sus etiquetas
    datos = [element.text for element in datos]

    #Creo una lista de listas donde guardaré los valores para cada fila que luego pasaré al Data Frame
    filas = [[]]

    #Hago un slicing por cada 10 elementos que son los valores para cada columna por fila
    for num in range(0,71, 10):
        filas.append(datos[num:num+10])

    df = pd.DataFrame(filas, columns = headers)
    return df
#     print(df)

#Obtengo el RSS de los cierres diarios de la BMV
def scrapping_RSS():
    url = "http://feeds.feedburner.com/BmvBoletin"

    rss = feedparser.parse(url)

    #Guardo cada una de las entradas existentes en el RSS
    rss_df = pd.DataFrame(rss.entries)

    #Ordeno las entradas para tener en el primer elemento la más reciente (el día anterior)
    rss_df_sorted = rss_df.sort_values("published", ascending = False)

    #Obtengo el cierre anterior que es un archivo HTML
    last_closed_link = rss_df_sorted["feedburner_origlink"][0]

    #Leo el archivo HTML del cierre del día anterior de la BMV
    html_file = requests.get(last_closed_link).content

    soup = BeautifulSoup(html_file)

    return scrapping(soup)


#######Funciones sobre el DataFrame######

def drop_duplicates(df):
    df = df.drop_duplicates()
    return df

def drop_na_rows(df):
    df = df.dropna()
    return df

def drop_na_columns(df):
    null_cols = df.isnull().sum()
    drop_cols = list(null_cols[null_cols > 10000].index)
    df = df.drop(drop_cols, axis=1)
    return df 

def data_cleaning(df):
    df = drop_duplicates(df)
    df = drop_na_rows(df)
    df = drop_na_columns(df)
    return df
    

#####Cálculo de información importante
def variacion_negativa(df):
    df["Variación"] = df["Variación"].astype(float)
    negativos = df[df["Variación"] < 0]
#     print(df.dtypes)
    return negativos["Índice", "Último", "Hora", "Anterior", "Variación", "Variación Ptos.", "Máximo", "Hora", "Mínimo", "Hora"]
    
    
df = scrapping_RSS()
df = data_cleaning(df)

#Obtener los índices con variación negativa
variacion_negativa(df)





,Índice,Último,Hora,Anterior,Variación,Variación Ptos.,Máximo,Hora,Mínimo,Hora
4,S&P/BMV FIBRAS,223.08,15:16,223.73,-0.29,-0.65,226.94,9:02,221.48,2:15
7,S&P/BMV IRT MidCap,347.71,15:16,348.29,-0.17,-0.58,351.24,9:02,346.83,9:46


API CHALLENGE

In [299]:
# ovgTp7SJufoqcjr6g2205dt0KATD49AIMVY4ht5B
# https://api.usa.gov/crime/fbi/sapi/
# api = "https://api.usa.gov/crime/fbi/sapi/api/data/arrest/national/offense/juvenile/2010/2019?api_key=ovgTp7SJufoqcjr6g2205dt0KATD49AIMVY4ht5B"


def get_response(start_date):
    start_date_api = datetime.strptime(start_date, "%Y-%m-%d").date()
    end_date = start_date_api + timedelta(days = 7)
    url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date_api}&end_date={end_date}&api_key=T0tZNACEDXLIyexmdmTCSCrMemQ0u39v1syrwXzJ"
    response = requests.get(url)
    asteroids_json = response.json()
    asteroids_normalized = json_normalize(asteroids_json)
    asteroids_df = pd.DataFrame(asteroids_normalized[f"near_earth_objects.{start_date}"][0])
    asteroids_return_df = asteroids_df[["name", "neo_reference_id", "is_potentially_hazardous_asteroid", "absolute_magnitude_h"]]
    return asteroids_return_df

   
get_response("2019-07-12")    

,name,neo_reference_id,is_potentially_hazardous_asteroid,absolute_magnitude_h
0,(2019 KD3),3842614,False,23.144
1,(2019 LT4),3842864,False,24.184
2,(2019 NW5),3843276,False,23.779
3,(2019 JC47),3842749,False,20.501
4,(2019 NC),3843081,False,24.295
5,(2016 LM48),3754252,False,20.500
6,(2016 NJ33),3755347,False,25.200
7,(2017 FK3),3771643,False,26.200
8,257744 (2000 AD205),2257744,False,18.500
9,293054 (2006 WP127),2293054,False,18.300
